# 1. Introduction

Bayesian Interaction Primitives (referred to as BIP from this point forward) are a human-robot interaction framework which focuses on extracting a probabilistic model of interaction dynamics as presented in a set of example demonstrations.
This section will provide a brief introduction as to what exactly this means as well as an intuitive description of how BIP goes about building its model (more in-depth mathematical details will be presented in Section 4).
A quickstart on how to get up-and-running with the library can be found in Section 2 while a thorough tutorial can be found in Section 3.

### What is human-robot interaction?

Loosely speaking, human-robot interaction (HRI) is any sort of physical or non-physical interaction in which a human and a robot extend some form of mutual influence or action to each other.
On it's own this is a massive field and can encompass interactions ranging from verbal communication to physical activity, and draws from fields such as natural language processing, computer vision, planning, control theory, and many more.
Even the social sciences play an important role, as humans have certain expectations when interacting with others, and robots must be aware of and fulfill these expectations in order to result in comfortable interactions.
In fact, colloquially speaking we consider something to be "robotic" precisely when it fails to fulfill these expectations.
So as roboticists, our goal should be to produce human-robot interactions which are anything but "robotic"!

![](media/examples_new.png)

With Bayesian Interaction Primitives, we focus specifically on cooperative physical interactions, for example, shaking hands or jointly lifting a large object.
Due to how we capture the dynamics of an interaction, BIP can actually be used to model *any* temporal multivariate observations which exhibit strong correlations, however, we will focus only on HRI scenarios in this documentation.

### What do we mean by interaction dynamics?

We define the dynamics of an interaction as how an interaction evolves over time, both in terms of the spatial relationship between interaction partners and the temporal relationship.
For example, in a handshake interaction, the hands of each partner converge in physical space and clasp for an amount of time that is not explicitly communicated, rather these are inherent to the *dynamics* of an interaction and are understood by both partners.
Modeling these dynamics allows us to reproduce the full interaction after only observing a part of it because we know the temporal and spatial relationship over time.
Fundamentally, this is what allows us to perform inference with Bayesian Interaction Primitives.

### How are the interaction dynamics modeled?

The key question that we are answering with Bayesian Interaction Primitives is: how do we model these dynamics?
Deriving an analytical model up front is extremely challenging given the inherent variability in human behavior.
Instead, we take a data-driven approach known as *imitation learning* or *learning-from-demonstration*, in which we observe example demonstrations of the interaction in question in order to extract the interaction dynamics.
This approach resembles how humans and animals learn new skills: through observation.

Before diving into the math in Section 4, we will begin by giving a high-level overview of how the models are actually built.


![](media/bip_overview.png)










##### Training
During training, we observe demonstrations of the interaction.
These interactions can either be performed by two humans (in which case the correspondence problem must be solved at some point if robot control is desired) or between a human and a robot (known as kinesthetic teaching).
Each interaction yields a sequence of observations over time for every measured degree of freedom (DoF).
However, these sequences are difficult to work with directly, so we decompose them into a latent space composed of a linear combination of nonlinear basis functions, where the coefficients represent the latent state of an interaction.
This allows us to construct a prior in the form of a joint probability distribution in the combined latent space of all DoFs, which represents the region where we believe a successful interaction lies.

##### Testing
During testing, we begin by observing a partial sequence of observations from a subset of the DoFs.
At this point, there are two unknowns: 1) what specific latent state corresponds to the current interaction, and 2) how much of the interaction have we observed.
These are related problems; the transformation between the latent space and observation space requires knowing how much of the interaction has occurred (the phase of the interaction).
In order to integrate the observations then, we perform inference in which we simultaneously infer both the spatial state of the interaction (the latent state) and the temporal state (the phase).
Inference involves updating our prior distribution based on these observations to produce a new posterior distribution from which we generate control trajectories for the robot.

However, since both the prior and posterior distributions are joint probability distributions over all DoFs, we can also infer future states for any DoF in the model.
For example, suppose the human partner provided demonstrations in which they wore pressure-sensitive shoes.
If these shoes are no longer worn at test time we can still infer what the values should be given the observations that we do have.
This provides a particularly powerful framework for dealing with sensor failures, be it from occlusion, noise, hardware failure, or otherwise.

##### What is IntPrim?

IntPrim is a Python library implementing Bayesian Interaction Primitives.
It provides the following features out of the box:

* Train a BIP model from demonstrations
* Perform recursive inference with a BIP model and generate future states
* Support for inference with ensemble Bayesian Interaction Primitives, Bayesian Interaction Primitives, Probabilistic Movement Primitives with DTW, and Particle Filter
* Automatic basis space selection with support for Gaussian, Sigmoidal, and Polynomial functions
* Automatic computation of the observation noise
* Comprehensive interactive analysis tools

The rest of this document will discuss these features and more.